# Databaser med Flask

pip install mysql-connector-python 
import mysql.connector

###om database connection: 
When connecting to a database using mysql.connector, the database needs to actually exist on a MySQL server - the SQL file alone is just a definition, not the database itself.

I'll explain how the database connection works in this code:

In the DataBase class constructor, the connection is established to:

A MySQL server running on "localhost" (your local machine)
Using username "root"
With password from environment variables
Connecting to a database named "dte_2509"
The SQL file (users.sql) is just a script that defines the structure of tables, but it needs to be executed against an existing MySQL server to create those tables.

I'll modify the database.py file to add comments that explain this:

database.py+10-1
To elaborate further on how the database system works in your project:

Database Setup and Connection Process
MySQL Server: The actual database system is MySQL, which needs to be installed and running on your computer ("localhost"). This is completely separate from your SQL definition files.

Database Creation: Before your code can work, you need to manually create a database named "dte_2509" on your MySQL server. This would typically be done with:

Table Creation: The users.sql file contains the SQL commands to create your tables, but it doesn't automatically execute. You need to run this file against your MySQL server:

Connection Parameters: Your DataBase class connects to this existing database using:

Host: localhost (your local machine)
User: root
Password: from an environment variable DB_PASSWORD
Database name: dte_2509
Connection Process: When you instantiate the DataBase class, mysql.connector opens a connection to the MySQL server running on your machine and selects the "dte_2509" database to work with.

The database isn't "in" the SQL file - the SQL file is just a script that defines the structure. The actual database exists as files managed by your MySQL server, typically stored in locations like /var/lib/mysql on Linux or C:\ProgramData\MySQL\MySQL Server X.X\Data on Windows, depending on your MySQL installation.

In [ ]:
import mysql.connector

connection = mysql.connector.connect(
    host="localhost"
    user="username"
    password="password" # tre sikre måter for passordhåndtering
    database="database" 
)

# cursor er objectet som brukes for kommunikasjon med db
cursor = connection.cursor()

# Example: SELECT query
cursor.execute("SELECT * FROM users WHERE age > %s", (25,))

# Fetch the results
results = cursor.fetchall()  # Fetch all rows
# Or
first_row = cursor.fetchone()  # Fetch a single row

# INSERT example
cursor.execute("INSERT INTO users (name, email) VALUES (%s, %s)", 
              ("John Doe", "john@example.com"))

# UPDATE example
cursor.execute("UPDATE users SET email = %s WHERE name = %s", 
              ("newemail@example.com", "John Doe"))

# DELETE example
cursor.execute("DELETE FROM users WHERE name = %s", ("John Doe",))

# Commit the changes
connection.commit()

# Always close cursor and connection when done
cursor.close()
connection.close()

#i dette faget lages db klasse for interaksjon med db:
# #__exit__ and __enter__ dunder methods must be override to use "with"
# purpose is to create autmoatic creasion of cursor and cleanup of reasources 
class DataBase():
    def __init__(self):
        self.mysqlConnection = mysql.connector.connect(
            host="localhost", #:3306
            user="root",
            password=os.getenv("DB_PASSWORD"),
            database="dte_2509"
        )

    def __enter__(self):
        try:
            self.cursor = self.mysqlConnection.cursor()
            return self
        except mysql.connector.Error as error:
            print("Error while connecting to MySQL", error)

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.mysqlConnection.commit()
        self.cursor.close()
        self.mysqlConnection.close()

# User managment

pip install flask-login

### load_user()
The load_user function is a crucial part of Flask-Login's functionality. Let me explain what it does and why it's required:

The load_user function is a callback that Flask-Login needs to retrieve a user object based on the user's ID. This is essential for maintaining user sessions and authentication state across requests.

Here's why it's required and what it does:

Session Management: Flask-Login stores the user's ID in the session when they log in. For every subsequent request, it needs to convert that ID back into the actual user object.

Authentication Persistence: Without this function, Flask-Login wouldn't be able to retrieve the currently logged-in user between requests.

Implementation: In your code, the function:

Takes a user_id parameter (which Flask-Login passes from the session)
Opens a database connection using your DataBase context manager
Calls the database's load_user method to retrieve the user's data
If a user is found, constructs and returns a User object with the data
Returns None if no user is found (which Flask-Login interprets as "no user logged in")
Required Decorator: The @login_manager.user_loader decorator registers this function with Flask-Login so it knows which function to call when it needs to load a user.

Without this function, features like @login_required and current_user wouldn't work properly, as Flask-Login wouldn't be able to determine if a user is authenticated or who the current user is.

In [ ]:
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'users.login' # When unauthorized redirect to 'login'